In [9]:
import pandas as pd
import numpy as np
import ast
import difflib
pd.options.display.max_rows = 50
pd.options.display.min_rows = 50

In [19]:
df = pd.read_csv('curses.csv', index_col=0)

In [20]:
df.head(5)

,name,price,ratio,time,skills,school,Сложность,Тип обучения,Формат обучения,Трудоустройство,Сертификат,Зач. единицы
0,Бухгалтер,81125 ₽,4.8,NaN,"['1С:Предприятие', 'КДП', 'Бухгалтерский учет'...",Skillbox,Начинающий,Профессия,С проверкой домашнего задания,Гарантия,Да,NaN
1,Дизайн среды: от интерьера до города,164700 ₽,4.8,"23 месяца,","['Sketch', 'Скетчинг', 'Визуализация', 'Дизайн...",Нетология,Начинающий,Курс,С проверкой домашнего задания,Гарантия,Да,NaN
2,Как правильно вести кадровое делопроизводство,6000 ₽,4.2,NaN,['КДП'],КонтурШкола,Начинающий,Курс,Записанные лекции,Отсутствует,NaN,NaN
3,Прикладной анализ данных в медицинской сфере,640000 ₽,4.8,2 года,"['Анализ данных', 'Биостатистика', 'Экономика ...",Московский физико-технический институт,Продвинутый,Высшее образование,С проверкой домашнего задания,Отсутствует,Да,NaN
4,Руководитель проектов в строительстве,134700 ₽,4.4,"12 месяцев по 2 занятия в неделю,","['Закупки', 'Нормативно-правовая база', 'Управ...",GeekBrains,Начинающий,Профессия,С проверкой домашнего задания,Гарантия,Да,NaN


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6190 entries, 0 to 6189
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             6190 non-null   object 
 1   price            6190 non-null   object 
 2   ratio            6190 non-null   float64
 3   time             5069 non-null   object 
 4   skills           6190 non-null   object 
 5   school           6190 non-null   object 
 6   Сложность        5883 non-null   object 
 7   Тип обучения     6190 non-null   object 
 8   Формат обучения  5950 non-null   object 
 9   Трудоустройство  5692 non-null   object 
 10  Сертификат       4862 non-null   object 
 11  Зач. единицы     760 non-null    float64
dtypes: float64(2), object(10)
memory usage: 628.7+ KB


In [22]:
def clear_price(data):
    
    if data == ' бесплатно ' or data == ' нет данных ' or data is np.nan:
        return np.nan
    tmp = int(data.split(' ')[0])
    if tmp < 1:
        return np.nan
    return tmp

df['price'] = df['price'].apply(clear_price)


In [23]:
df['skills'] = df['skills'].apply(lambda x : ast.literal_eval(x))

In [24]:
n = 16
def rename(data):
    sq = difflib.SequenceMatcher(None, data, "")
    for name in names:
        if len(data) <= len(name):
            continue
        sq.set_seq2(name)
        if (sq.find_longest_match()[2] > n):
            data = name
            sq.set_seq1(data)
    return data

names = pd.Series(df['name'].unique())
names = names[names.str.len() > n]
df['name'] = df['name'].apply(rename)

In [25]:
def clear_time(data):
    if data is np.nan:
        return np.nan
    result = []
    for i in data.strip(',').split():
        if 'год' in i:
            result.append(' * 8760 +')
            if i[-1] == ',':
                break
        elif 'мес' in i:
            result.append(' * 732 +')
            if i[-1] == ',':
                break
        elif 'нед' in i:
            result.append(' * 168 +')
            if i[-1] == ',':
                break
        elif 'дн' in i:
            result.append(' * 24 +')
            if i[-1] == ',':
                break
        elif 'час' in i:
            result.append(' * 1 +')
            if i[-1] == ',':
                break
        elif 'мин' in i:
            result.append(' / 60 +')
            if i[-1] == ',':
                break
        elif (i == 'по') | ('ак' in i) | ('уро' in i):
            break
        elif (',' in i):
            result.append(i.replace(',','.'))
        else:
            result.append(i)
    result = ''.join(result)
    try:
        if result[-1] == '+':
            result = eval(result[:-1])
        else:
            result = eval(result)
    except:
        result = np.nan
    return result if result >= 0 else np.nan
df['time'] = df['time'].apply(clear_time)

In [26]:
skills_dict = {'sql': 'SQL', 'git': 'Git', 'linux': 'linux', 'python': 'Python', 'java script': 'Java Script',
               'javascript': 'Java Script', 'json': 'JSON', 'js': 'Java Script', 'xml': 'XML', '1c': '1C',
               '1 c': '1C', '1с': '1C', '1 с': '1C', 'html': 'HTML', 'css': 'CSS', 'c#': 'C#', 'с#': 'C#',
               'php': 'PHP', 'docker': 'Docker', 'rest': 'REST', 'nix': 'Nix', 'ооп': 'ООП', 'jira': 'Atlassian Jira ',
               'с++': 'C++', 'c++': 'C++', 'android': 'Android', 'swift': 'Swift/iOS', 'ios': 'Swift/iOS',
               'kotlin': 'Kotlin', 'tcp': 'TCP/IP', 'информационные технологии': 'Информационные технологии',
               'информационная безопасность': 'Информационная безопасность',
               'информационной безопасности': 'Информационная безопасность',
               'dos': 'DOS', 'net': '.NET',
               'api': 'API', 'active directory': 'Active directory', 'jquery': 'JQuery',
               'настройка пк': 'Настройка ПК', 'настройка сетевых подключений': 'Настройка сетевых подключений',
               'сетевые технологии': 'Настройка сетевых подключений',
               'настройка по': 'Настройка ПО', 'установка по': 'Настройка ПО',
               'windows': 'Windows', 'базам': 'Работа с БД', 'базы дан': 'Работа с БД',
               'субд': 'Работа с БД',
               'техническая поддержка': 'Техническая поддержка', 'сетевог': 'Настройка сетевого оборудования',
               'crm': 'Crm',
               'react': 'React', 'typescript': 'TypeScript', 'dhcp': 'DHCP',
               'анализ': 'Анализ данных', 'bash': 'Bash', 'zabbix': 'Zabbix', 'laravel': 'Laravel',
               'dns': 'Dns', 'cisco': 'Cisco', 'bpmn': 'BPMN', 'тестирование': 'Тестирование',
               'nginx': 'Nginx', 'kubernetes': 'Kubernetes', 'vmware': 'VMware', 'redis': 'Redis',
               'постановка задач разработчикам': 'Постановка задач разработчикам', 'mongodb': 'MongoDB',
               'scrum': 'SCRUM', 'программирование': 'Программирование',
               'защиты инфор': 'Cредства информационной защиты',
               'uml': 'UML', 'разработка по': 'Разработка ПО', 'spring': 'Spring FrameWork',
               'symfony': 'Symfony', 'cms': 'CMS', 'ansible': 'Ansible', 'django': 'Django FrameWork',
               'agile': 'AGILE', 'bootstrap': 'BootStrap', 'локаль': 'Администрирование локальных сетей',
               'unity': ' Unity',
               'rabbit': 'Rabbit', 'vpn': 'VPN', 'hyper-v': 'Hyper-v',
               'ajax': 'AJAX', 'vue': 'Vue', 'ci/cd': 'CI/CD',
               'golang': 'Golang', 'mvc': 'MVC', 'oracle': 'Oracle',
               'битрикс': 'Bitrix', 'bitrix': 'Bitrix', 'visual studio': 'Visual Studio',
               'soap': 'SOAP', 'mac ': 'Mac OS', 'qt': 'QT',
               'atlassian confluence': 'Atlassian Confluence', 'http': 'HTTP', 'верст': 'Верстка',
               'сервер': 'Настройка серверов', 'системный администратор': 'Системный администратор',
               'powershell': 'PowerShell',
               'yii': 'Yii', 'access': 'MS Access', 'kafka': 'kafka',
               'delphi': 'Delphi', 'sass': 'SASS', 'solid': 'SOLID',
               'qa': 'QA', 'jenkins': 'Jenkins', 'devops': 'DevOps', 'webpack': 'Webpack',
               'backend': 'Backend', 'aws': 'Aws', 'office': 'MS Office', 'asterisk': 'Asterisk',
               'hibernate': 'Hibernate', 'grafana': 'Grafana', 'mvvm': 'MVVM', 'centos': 'CentOS',
               'power bi': 'Power BI', 'powerbi': 'Power BI', 'wpf': 'WPF', 'git': 'Git',
               'test': 'Testing', 'flutter': 'Flutter', 'prometheus': 'Prometheus', 'метрик': 'Метрики',
               'entity': 'Entity FrameWork', 'gamedev': 'GameDev', 'redmine': 'Redmine', 'osi': 'OSI',
               'антивирус': 'Антивирусная защита', 'сетевого': 'Настройка сетевого оборудования',
               'frontend': 'Frontend'
               }


def skills_transformation(data):
    result = set()
    for skill in data:
        skill = skill.lower()
        for key in skills_dict.keys():
            if key in skill:
                result.add(skills_dict[key])
                skill = skill.replace(key, '')
        if skill == 'go':
            result.add('Golang')

    return tuple(result) if result else np.nan


df['skills'] = df['skills'].apply(skills_transformation)


In [28]:
df.head(5)

,name,price,ratio,time,skills,school,Сложность,Тип обучения,Формат обучения,Трудоустройство,Сертификат,Зач. единицы
0,Бухгалтер,81125.0,4.8,NaN,"(1C,)",Skillbox,Начинающий,Профессия,С проверкой домашнего задания,Гарантия,Да,NaN
1,Дизайн среды: от интерьера до города,164700.0,4.8,16836.0,NaN,Нетология,Начинающий,Курс,С проверкой домашнего задания,Гарантия,Да,NaN
2,Конфиденциальное делопроизводство,6000.0,4.2,NaN,NaN,КонтурШкола,Начинающий,Курс,Записанные лекции,Отсутствует,NaN,NaN
3,Обработка и анализ данных в SQL,640000.0,4.8,17520.0,"(Python, Анализ данных)",Московский физико-технический институт,Продвинутый,Высшее образование,С проверкой домашнего задания,Отсутствует,Да,NaN
4,Руководитель проектов в строительстве,134700.0,4.4,8784.0,"(SCRUM, AGILE)",GeekBrains,Начинающий,Профессия,С проверкой домашнего задания,Гарантия,Да,NaN


In [29]:
df[['name', 'price', 'time', 'skills', 'school', 'ratio']] \
        .dropna() \
        .reset_index(drop=True) \
        .explode('skills') \
        .to_csv(f'curses_for_pbi.csv', header=['name','price','time','skills','school', 'ratio'])

In [31]:
df.to_csv("curses_clear.csv")